## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bartica,GY,2021-11-06 16:54:13,6.4000,-58.6167,77.32,92,100,4.38,moderate rain
1,1,Adrar,MR,2021-11-06 16:54:13,20.5022,-10.0711,86.76,18,12,9.57,few clouds
2,2,Iranshahr,IR,2021-11-06 16:54:14,27.2025,60.6848,79.14,24,0,0.00,clear sky
3,3,Rikitea,PF,2021-11-06 16:54:14,-23.1203,-134.9692,75.18,81,100,16.49,overcast clouds
4,4,Atuona,PF,2021-11-06 16:54:15,-9.8000,-139.0333,77.99,75,39,16.87,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Adrar,MR,2021-11-06 16:54:13,20.5022,-10.0711,86.76,18,12,9.57,few clouds
26,26,Teseney,ER,2021-11-06 16:54:25,15.1100,36.6575,89.64,22,10,8.39,clear sky
30,30,Beira,MZ,2021-11-06 16:54:27,-19.8436,34.8389,80.69,78,0,8.99,clear sky
45,45,Cabo San Lucas,MX,2021-11-06 16:54:34,22.8909,-109.9124,85.51,53,58,1.66,broken clouds
49,49,Nagua,DO,2021-11-06 16:54:36,19.3832,-69.8474,89.46,70,20,11.50,few clouds
53,53,Oriximina,BR,2021-11-06 16:54:39,-1.7656,-55.8661,88.84,53,86,4.72,overcast clouds
54,54,Hithadhoo,MV,2021-11-06 16:54:40,-0.6000,73.0833,82.40,75,100,13.24,overcast clouds
57,57,Urucui,BR,2021-11-06 16:54:42,-7.2294,-44.5561,90.54,49,76,4.14,broken clouds
67,67,Nhulunbuy,AU,2021-11-06 16:54:46,-12.2333,136.7667,81.21,83,21,4.61,few clouds
72,72,Topolobampo,MX,2021-11-06 16:54:48,25.6000,-109.0500,82.06,39,83,1.72,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                134
City                   134
Country                132
Date                   134
Lat                    134
Lng                    134
Max Temp               134
Humidity               134
Cloudiness             134
Wind Speed             134
Current Description    134
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                132
City                   132
Country                132
Date                   132
Lat                    132
Lng                    132
Max Temp               132
Humidity               132
Cloudiness             132
Wind Speed             132
Current Description    132
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Adrar,MR,86.76,few clouds,20.5022,-10.0711,
26,Teseney,ER,89.64,clear sky,15.1100,36.6575,
30,Beira,MZ,80.69,clear sky,-19.8436,34.8389,
45,Cabo San Lucas,MX,85.51,broken clouds,22.8909,-109.9124,
49,Nagua,DO,89.46,few clouds,19.3832,-69.8474,
53,Oriximina,BR,88.84,overcast clouds,-1.7656,-55.8661,
54,Hithadhoo,MV,82.40,overcast clouds,-0.6000,73.0833,
57,Urucui,BR,90.54,broken clouds,-7.2294,-44.5561,
67,Nhulunbuy,AU,81.21,few clouds,-12.2333,136.7667,
72,Topolobampo,MX,82.06,broken clouds,25.6000,-109.0500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping")  

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()

In [13]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map.

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.6)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure.
fig

Figure(layout=FigureLayout(height='420px'))